In [29]:
import torch

In [30]:
from ops.v_models import VNet, v_TSN

In [31]:
from ops.models import TSN

In [37]:
num_class = 174
model = TSN(num_class, 8, "RGB",
            base_model="resnet50",
            consensus_type="avg",
            dropout=0.5,
            img_feature_dim=256,
            partial_bn=True,
            is_shift=False, shift_div=8, shift_place="blockres",
            fc_lr5=False,
            temporal_pool=False,
            non_local=False)
v_model = v_TSN(num_class, 3, "RGB",
            base_model="resnet50",
            consensus_type="avg",
            dropout=0.5,
            img_feature_dim=256,
            partial_bn=True,
            is_shift=False, shift_div=8, shift_place="blockres",
            fc_lr5=False,
            temporal_pool=False,
            non_local=False)
vnet = VNet(1, 100, 1)


    Initializing TSN with base model: resnet50.
    TSN Configurations:
        input_modality:     RGB
        num_segments:       8
        new_length:         1
        consensus_module:   avg
        dropout_ratio:      0.5
        img_feature_dim:    256
            
=> base model: resnet50

    Initializing TSN with base model: resnet50.
    TSN Configurations:
        input_modality:     RGB
        num_segments:       3
        new_length:         1
        consensus_module:   avg
        dropout_ratio:      0.5
        img_feature_dim:    256
            
=> base model: resnet50


In [38]:
criterion = torch.nn.CrossEntropyLoss()
valcriterion = torch.nn.CrossEntropyLoss()

In [39]:
model.train()
v_model.load_state_dict(model.state_dict())

Freezing BatchNorm2D except the first one.


<All keys matched successfully>

In [40]:
input_var = torch.randn((4, 24, 224, 224), requires_grad=True)
input_var.shape

torch.Size([4, 24, 224, 224])

In [41]:
output = v_model(input_var)
cost = criterion(output, target_var)

RuntimeError: shape '[-1, 3, 174]' is invalid for input of size 5568

In [ ]:
cost_v = torch.reshape(cost, (-1, 1))
v_lambda = vnet(cost_v.data)
l_f_v = torch.sum(cost_v * v_lambda) / len(cost_v)
v_model.zero_grad()
grads = torch.autograd.grad(l_f_v, (v_model.module.params()), create_graph=True)
v_lr = 0.01
v_model.module.update_params(lr_inner=v_lr, source_params=grads)
del grads